<a href="https://colab.research.google.com/github/mrorvelle/ErlangC-Calculation/blob/master/textAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Set up way to extract mp3 from a website and convert to .wav

# Access Azure DB

In [240]:
!pip install mysql.connector
!pip3 install SpeechRecognition pydub
import mysql.connector
from mysql.connector import errorcode
import pandas as pd
import speech_recognition as sr 
import os 
from pydub import AudioSegment
from pydub.silence import split_on_silence

#[!INCLUDE[applies-to-mysql-single-server](includes/applies-to-mysql-single-server.md)]
config = {
  'host':'sermons.mysql.database.azure.com',
  'user':'trevorpollo@sermons',
  'password':'PITW78twias!',
  'database':'sermons',
  #'client_flags': [mysql.connector.ClientFlag.SSL],
  #'ssl_ca': '/var/wwww/html/DigiCertGlobalRootG2.crt.pem'
}

# Construct connection string

#[!INCLUDE[applies-to-mysql-single-server](includes/applies-to-mysql-single-server.md)]
try:
   conn = mysql.connector.connect(**config)
   print("Connection established")
except mysql.connector.Error as err:
  if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
    print("Something is wrong with the user name or password")
  elif err.errno == errorcode.ER_BAD_DB_ERROR:
    print("Database does not exist")
  else:
    print(err)
else:
  cursor = conn.cursor()

#cursor.execute("CREATE DATABASE sermons;")

Processing /root/.cache/pip/wheels/8c/83/a1/f8b6d4bb1bd6208bbde1608bbfa7557504bed9eaf2ecf8c175/mysql_connector-2.2.9-cp37-cp37m-linux_x86_64.whl
Connection established


# Function to process audio file

In [241]:
def get_large_audio_transcription(path, df):
          """
          Splitting the large audio file into chunks
          and apply speech recognition on each of these chunks
          """

          # open the audio file using pydub
          sound = AudioSegment.from_wav(path)  
          # split audio sound where silence is 700 miliseconds or more and get chunks
          chunks = split_on_silence(sound,
              # experiment with this value for your target audio file
              min_silence_len = 750,
              # adjust this per requirement
              silence_thresh = sound.dBFS-14,
              # keep the silence for 1 second, adjustable as well
              keep_silence=1000,
          )
          folder_name = "audio-chunks"
          # create a directory to store the audio chunks
          if not os.path.isdir(folder_name):
              os.mkdir(folder_name)
          whole_text = ""
          # process each chunk 
          for i, audio_chunk in enumerate(chunks, start=1):
              # export audio chunk and save it in
              # the `folder_name` directory.
              chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
              audio_chunk.export(chunk_filename, format="wav")
              # recognize the chunk
              with sr.AudioFile(chunk_filename) as source:
                  audio_listened = r.record(source)
                  # try converting it to text
                  try:
                      text = r.recognize_google(audio_listened)
                  except sr.UnknownValueError as e:
                      print("Error:", str(e))
                  else:
                      text = f"{text.capitalize()}. "
                      #print(chunk_filename, ":", text)
                      whole_text += text
                      df = df.append({'church' : church, 'date' : sermon, 'pastor' : pastor, 'text' : text}, 
                      ignore_index = True)
          
          !pip install pymysql
          import sqlalchemy as sal
          from sqlalchemy import create_engine

          engine = create_engine('mysql+pymysql://trevorpollo@sermons:PITW78twias!@sermons.mysql.database.azure.com:3306/sermons', echo=False)
          cnx = engine.raw_connection()
          data = df
          data.to_sql(name='sermons', con=engine, if_exists = 'append', index=False)

          import numpy as np

          #keyword search
          abortionarray = np.array(['abortion', 'abort', 'prolife', 'prochoice', 'pro life', 'pro choice', 'abort', 'fetus', 'unborn', 'preborn', 'embryo', 'life begin', 'planned pa','pregnancy', 'pregnant', 'pregna', 'conception', 'conceived', 'unplanned'])
          racearray = np.array(['social jutice', 'equality', 'racism', 'race', 'bigotry', 'discrimina', 'prejudice', 'slavery', 'segregation', 'systemic', 'hateful', 'black lives', 'brutality', 'militia'])
          sexsinarray = np.array(['porn', 'adultery', 'cheat', 'rape', 'modesty', 'modest', 'sex', 'fetish', 'grope', 'masturba', 'lust', 'polygamy', 'gay', 'lesbian', 'lgb', 'queer', 'homo', 'homos', 'marriage', 'pre mar', 'sodom', 'pervert'])
          politicsarray = np.array(['democrat', 'republican', 'politic', 'elected', 'president', 'vote', 'congress', 'war', 'equality', 'equity', 'constitution', 'independence', 'freedom', 'rights', 'liberty', 'equality', 'feminis'])
          gayarray = np.array(['gay', 'lesbi', 'homos', 'homo', 'same sex', 'attraction', 'lgb', 'rainbow', 'queer'])
          feministarray = np.array(['complimen', 'household', 'femini', 'role', 'head of h', 'spiritual le', 'male', 'female', 'gender', 'women', 'men'])

          #the resulting counts will go into a sql table with the key being concat('DateChurchPastor')
          abortion_counter = 0
          race_counter = 0
          sexsin_counter = 0
          political_counter = 0
          gay_counter = 0
          feminism_counter = 0

          #create np array for faster processing
          dfarray = np.array(df.text)

          #loop through text of a sermon for keywords
          for i in dfarray:
            if any(ele in i for ele in abortionarray) == True:
              abortion_counter += 1
            elif any(ele in i for ele in racearray) == True:
              race_counter += 1
            elif any(ele in i for ele in sexsinarray) == True:
              sexsin_counter += 1
            elif any(ele in i for ele in politicsarray) == True:
              political_counter += 1
            elif any(ele in i for ele in gayarray) == True:
              gay_counter += 1
            elif any(ele in i for ele in feministarray) == True:
              feminism_counter += 1

          print('abortion refs ' + str(abortion_counter))

          if abortion_counter
          

          
          resultingdata = {'sermondate':[sermon], 'sermonchurch':[church], 'sermonpastor':[pastor], \
                           'abortioncount':[abortion_counter], 'race':[race_counter], 'sex':[sexsin_counter], 'feminism':[feminism_counter], 'gay':[gay_counter], 'political':[political_counter]}
          df2 = pd.DataFrame(resultingdata)
          engine = create_engine('mysql+pymysql://trevorpollo@sermons:PITW78twias!@sermons.mysql.database.azure.com:3306/sermons', echo=False)
          cnx = engine.raw_connection()
          data = df2
          data.to_sql(name='sermonstats', con=engine, if_exists = 'append', index=False)


          # return the text for all chunks detected
          #return whole_text
          return pd.DataFrame(df)

         

In [ ]:
import requests 
from os import path
from pydub import AudioSegment
import pandas as pd
import speech_recognition as sr 
import os 
from pydub import AudioSegment
from pydub.silence import split_on_silence

from csv import reader
# skip first line i.e. read header first and then iterate over each row od csv as a list
with open('/content/drive/MyDrive/sermonlists.csv', 'r') as read_obj:
    sermonsdf = pd.read_csv('/content/drive/MyDrive/sermonlists.csv')
    #print(sermonsdf.dtypes)
    # Check file as empty
        # Iterate over each row after the header in the csv
    for row in sermonsdf.itertuples():
      file_url = str(row.path)
      r = requests.get(file_url, stream = True) 
      with open("file_url.mp3", "wb") as file: 
        for block in r.iter_content(chunk_size = 1024):
              if block: 
                  file.write(block)
      src = 'file_url.mp3'
      dst = "/content/drive/MyDrive/"+str(row.date)+".wav"

      # convert wav to mp3                                                            
      sound = AudioSegment.from_mp3(src)
      sound.export(dst, format="wav")

      !touch dst
      os.remove(src)
      
      
      path = dst
      pastor = row.pastor
      church = row.church
      sermon = row.date
      df = pd.DataFrame(columns = ['church','pastor','date','text'])



      !pip3 install SpeechRecognition pydub
      # importing libraries 

      



      # create a speech recognition object
      r = sr.Recognizer()


      # a function that splits the audio file into chunks
      # and applies speech recognition
      
      df = get_large_audio_transcription(path, df)
   
